In [32]:
import boto3
import json
import urllib.parse
import urllib.request
import pandas as pd

In [33]:
def get_data(total_num, term='mcdonald'):
    num = 0
    offset = 0
    
    while num < total_num:
        param = {"term": term, "location": "Manhattan", "offset": offset, "limit": 50}
        param = urllib.parse.urlencode(param)
        url = 'https://api.yelp.com/v3/businesses/search'
        url = "?".join([url, param])
        # print(url)
        # print(num)
        headers = {"Authorization": ""
                                    ""}
        req = urllib.request.Request(url, headers=headers)
        response = urllib.request.urlopen(req)
        data = response.read()
        data = json.loads(data)
        if not data["businesses"]:
            break
        for item in data["businesses"]:
            if item["id"] in restaurant_id:
                continue
            elif item["name"] != "McDonald's":
                continue
            elif int(item["location"]["zip_code"]) < 10000:
                continue
            else:
                restaurant_id.append(item["id"])
                names.append(item["name"])
                address.append(item["location"]["address1"])
                num_of_reviews.append(item["review_count"])
                rating.append(item["rating"])
                zip_code.append(int(item["location"]["zip_code"]))
                num += 1
                if num == total_num:
                    break
        offset += 50
        if offset > 950:
            break
    print('retrived ',num,'results')


In [34]:
restaurant_id = []
names = []
num_of_reviews = []
rating = []
address = []
zip_code = []

In [35]:
get_data(15)

retrived  15 results


In [36]:
df = pd.DataFrame({"restaurant_id":restaurant_id,"names":names,
                   "num_of_reviews":num_of_reviews,"rating":rating,"zip_code":zip_code,"address":address})

In [37]:
df

,restaurant_id,names,num_of_reviews,rating,zip_code,address
0,aF05dAZpGr8eBiI2IkAFaA,McDonald's,65,2.0,10022,824 3rd Avenue 50th St
1,Fnmdad7dvjqwrEmQby12CQ,McDonald's,50,2.5,10022,966 3rd Ave
2,RvSqiP7Sj0kebrllKdDo_Q,McDonald's,64,2.0,10023,2049 Broadway
3,VqoXgaS-vTfCu-yrlYol1g,McDonald's,111,1.5,10021,1286 1st Ave
4,yaDp_eIhkgfT3SIn0tRsRw,McDonald's,75,2.5,10019,946 8th Ave
5,f0VJirNbBXsS8Ihvv-Rs1g,McDonald's,75,2.5,10019,1651 Broadway
6,oGq_6DQ1AoQrAYfFSDV_-Q,McDonald's,36,2.5,10016,480 3rd Ave
7,aPUz7avK_yxMsAkhKRwJZA,McDonald's,30,2.0,10017,14 E 47th St
8,OBTZAyTcj5xptQBl2mJTSA,McDonald's,57,2.0,10017,18 E 42nd St
9,nqcJCwOfkttQbxKBFGMHvQ,McDonald's,78,2.0,10028,1499 3rd Ave


In [38]:
print(type(zip_code))

<class 'list'>


In [39]:
import psycopg2

In [40]:
conn = psycopg2.connect(database='proj1part2',user='',password='',host='')
cur = conn.cursor()

In [41]:
for i in range(len(restaurant_id)):
    cur.execute("""INSERT INTO "Restaurants" VALUES(%s,%s,%s,%s,%s,%s);""",\
                (restaurant_id[i],zip_code[i],rating[i],names[i],num_of_reviews[i],address[i]))

In [30]:
# cur.execute('DELETE FROM "Restaurants";')

In [42]:
conn.commit()

In [125]:
a = cur.execute('SELECT * FROM "restaurants";')